In [1]:
import rasterio
import numpy as np
import pandas as pd
import os


In [32]:
def geotiff_to_dataframe(geotiff_path):
    with rasterio.open(geotiff_path) as src:
        data = src.read(1)  # Reading the first band
        nodata = src.nodata

        # Extracting coordinates and values
        pixel_values = data.flatten()
        rows, cols = np.indices(data.shape)
        x_coords, y_coords = rasterio.transform.xy(src.transform, rows, cols, offset='center')
        x_coords = np.array(x_coords).flatten()
        y_coords = np.array(y_coords).flatten()
        
        varname = geotiff_path.split('_')[0]

        # Creating DataFrame
        df = pd.DataFrame({
            'x_coord': x_coords,
            'y_coord': y_coords,
            # 'pixel_value': pixel_values
            # f'{geotiff_path}':pixel_values
            f'{varname}':pixel_values
        })
        # df = df[df['pixel_value'] != nodata]  # Removing NoData values
        # df = df[df[f'{geotiff_path}'] != nodata]  # Removing NoData values
        df = df[df[f'{varname}'] != nodata]  # Removing NoData values
        return df

In [33]:
def get_filenames_with_string(directory_path, search_string):
    matching_filenames = []
    for filename in os.listdir(directory_path):
        if search_string in filename:
            matching_filenames.append(filename)
    return [directory_path + fn for fn in matching_filenames]


In [35]:


directory_path = 'EarthEngineData/'
# all_dfs = []

# for filename in os.listdir(directory_path):
#     if filename.endswith('.tif'):
#         df = geotiff_to_dataframe(os.path.join(directory_path, filename))
#         all_dfs.append(df)

# combined_df = pd.concat(all_dfs, ignore_index=True)

pd.concat(list(map(geotiff_to_dataframe, get_filenames_with_string('EarthEngineData/', 'Temperature')))).describe()

,x_coord,y_coord,EarthEngineData/Temperature
count,280000.000000,280000.000000,274964.000000
mean,89.984242,34.037166,281.319144
std,11.557451,6.359984,11.233781
min,69.974269,23.001363,251.367224
25%,79.979256,28.559689,271.206523
50%,89.984242,34.037166,282.258041
75%,99.989228,39.514644,289.459022
max,109.994215,45.072969,300.759527


In [ ]:
print('i started around 1223 and im lazy so i didnt actually timeit but now im going to lunch')

In [ ]:
combined_df

In [ ]:
combined_df.to_csv('combined_data.csv', index=False)
